In [9]:
import os 
import numpy as np
import h5py as h5
import nrrd
import fnmatch

In [32]:
def readH5(file_path,**kwargs):
    """
    generic function to read a h5 file with one 
    subgroup elements by given filepath in a Dictonary 
    """

    import h5py as h5
    
    out = {}

    with h5.File(file_path, 'r') as f:

        for key in f.keys():

            if isinstance(f[key],h5.Group):
                out[key] = {}
                for sub_key in f[key]:
                    out[key][sub_key] = f[key][sub_key][...]
            else:
                out[key] = f[key][:]
        f.close()
    return out

In [10]:
src_path = "/Volumes/LaCie/scratch/"

In [57]:
found_first_item = False
for root, dirs, files in os.walk(src_path):

    for dir_name in dirs:
        # print(dir_name)
        if "rand_subvolume" in dir_name.lower():
            # print(dir_name)
            print(os.listdir(os.path.join(root,dir_name)))
            for file in fnmatch.filter(os.listdir(os.path.join(root,dir_name)),"*.h5"):
                
                print(os.path.join(root,dir_name,file))
                file_path = os.path.join(root,dir_name,file)

                data = readH5(file_path)

                data_type = data["raw"].dtype
                size = data["raw"].shape
                spacing = ( data["volume_data"]["VoxelSizeX"],
                            data["volume_data"]["VoxelSizeX"],
                            data["volume_data"]["VoxelSizeX"]
                            )

                space_origin = data["volume_data"]["Origin"]*spacing
                print(data_type,len(size),spacing,space_origin)


                header = create_nrrd_header(size, spacing, space_origin, data_type)
                fileout = os.path.splitext(file)[0]


                nrrd.write(os.path.join(root,dir_name,fileout + ".nrrd"), data["raw"],header=header)
                


                found_first_item = False
                
        if found_first_item:
            break
    if found_first_item:
            break        

[]
['160_10-layer_00_0.h5', '160_10-layer_00_1.h5', '160_10-layer_00_0.nrrd', '160_10-layer_00_1.nrrd']
/Volumes/LaCie/scratch/160_10-layer/volumes/00/rand_subvolume/160_10-layer_00_0.h5
uint16 3 (array(0.020618), array(0.020618), array(0.020618)) [ 0.        9.236864 17.154176]
/Volumes/LaCie/scratch/160_10-layer/volumes/00/rand_subvolume/160_10-layer_00_1.h5
uint16 3 (array(0.020618), array(0.020618), array(0.020618)) [ 0.       31.669248 40.906112]
['160_10-layer_01_0.h5', '160_10-layer_01_1.h5', '160_10-layer_01_0.nrrd']
/Volumes/LaCie/scratch/160_10-layer/volumes/01/rand_subvolume/160_10-layer_01_0.h5
uint16 3 (array(0.020618), array(0.020618), array(0.020618)) [ 0.       25.071488 23.751936]
/Volumes/LaCie/scratch/160_10-layer/volumes/01/rand_subvolume/160_10-layer_01_1.h5
uint16 3 (array(0.020618), array(0.020618), array(0.020618)) [ 0.       10.556416 21.112832]
['160_10-layer_02_0.h5', '160_10-layer_02_1.h5', '160_10-layer_02_0.nrrd']
/Volumes/LaCie/scratch/160_10-layer/volume

In [42]:
def create_nrrd_header(size, spacing,space_origin, data_type='float', space='left-posterior-superior'):
    header = {
        'type': data_type,
        'dimension': len(size),
        'space dimension': len(size),
        'sizes': size,
        'space': space,
        'space origin': space_origin
    }

    if len(spacing) == len(size):
        header['space directions'] = np.diag(spacing).tolist()
        header['kinds'] = ['domain'] * len(size)

    header.update({
        'endian': 'little',
        'encoding': 'raw',
    })

    return header

In [40]:
def numpy_dtype_to_nrrd_dtype(dtype):
    """
    Convert NumPy dtype to NRRD dtype string.

    Parameters:
    - dtype: NumPy dtype object

    Returns:
    - NRRD dtype string
    """
    dtype_mapping = {
        np.uint8: 'uint8',
        np.uint16: 'uint16',
        np.uint32: 'uint32',
        np.uint64: 'uint64',
        np.int8: 'int8',
        np.int16: 'int16',
        np.int32: 'int32',
        np.int64: 'int64',
        np.float16: 'float16',
        np.float32: 'float32',
        np.float64: 'float64',
    }

    return dtype_mapping.get(dtype, 'unknown')